<a href="https://colab.research.google.com/github/MLo7Ghinsan/MLo7-colab-notebook/blob/main/so_vits_svc_notebook_mlo7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SO-VITS-SVC NOTEBOOK**

You must read and acknowledge the **Terms of Use** in order to understad the guideline.

Notebook made and maintained MLo7
_Notebook updated on: 3/28/2023_

Update log:
+ test lauch notebook

# Terms of Use
##### ~~copied from the main repo~~

### Please solve the authorization problem of the dataset on your own. You shall be solely responsible for any problems caused by the use of non-authorized datasets for training and all consequences thereof.The repository and its maintainer, svc develop team, have nothing to do with the consequences!

1. This project is established for academic exchange purposes only and is intended for communication and learning purposes. It is not intended for production environments.
2. Any videos based on sovits that are published on video platforms must clearly indicate in the description that they are used for voice changing and specify the input source of the voice or audio, for example, using videos or audios published by others and separating the vocals as input source for conversion, which must provide clear original video or music links. If your own voice or other synthesized voices from other commercial vocal synthesis software are used as the input source for conversion, you must also explain it in the description.
3. You shall be solely responsible for any infringement problems caused by the input source. When using other commercial vocal synthesis software as input source, please ensure that you comply with the terms of use of the software. Note that many vocal synthesis engines clearly state in their terms of use that they cannot be used for input source conversion.
4. Continuing to use this project is deemed as agreeing to the relevant provisions stated in this repository README. This repository README has the obligation to persuade, and is not responsible for any subsequent problems that may arise.
5. If you distribute this repository's code or publish any results produced by this project publicly (including but not limited to video sharing platforms), please indicate the original author and code source (this repository).
6. If you use this project for any other plan, please contact and inform the author of this repository in advance. Thank you very much.


# **Training Section**

In [ ]:
import os
from IPython.display import clear_output
from google.colab import drive
from IPython.display import Audio, display, HTML

drive.mount("/content/drive", force_remount=True)

if not os.path.exists("/content/play_sound"):
    os.makedirs("/content/play_sound")
%cd /content/play_sound
!wget -O setup_complete.wav https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/audio/setup_complete.wav

#@title #1.0 | SETUP | Install Dependencies and Mount Google Drive

%cd /content
!rm -rf /content/sample_data
!apt-get update
!apt-get install aria2
!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.0
%cd /content/so-vits-svc
!pip install --upgrade pip setuptools numba numpy
!pip install pyworld praat-parselmouth fairseq tensorboardX numba
!aria2c -x 16 -s 16 -j 16 --dir="/content/so-vits-svc/hubert" https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/diff-svc-necessary-checkpoints/checkpoint_best_legacy_500.pt

clear_output()

print("setup complete!")
print("|")
print("|")
print("|")

chika_dance = '<img src="https://cdn.discordapp.com/attachments/816517150175920138/1090112497446563950/icegif-2013.gif"/>'
display(HTML(chika_dance))

with open("/content/play_sound/setup_complete.wav", "rb") as f:
    setup_complete_sound = f.read()
Audio(data=setup_complete_sound, autoplay=True)

In [ ]:
#@title #2.0 | Extract data | Resume training from checkpoint
%cd /content
#@markdown ___
#@markdown ###Train from scratch section
#@markdown +=========================+
#@markdown ####Directory of the zip file that contain all of your recordings that you want to use to train a model
train_from_scratch = False #@param {type:"boolean"}
raw_data_zip_path = "path-to-the-zip-file-of-your-recordings"  #@param {type:"string"}
model_name = "your-model-name" #@param {type:"string"}

#@markdown ___
#@markdown ###Resume training section
#@markdown +=======================+
#@markdown ####Directory of the folder that contain your latest G_#, and D_# file
resume_training_from_checkpoint = False #@param {type:"boolean"}
#@markdown Directory of the zip file that THIS NOTEBOOK saved, or any zip that is in the same structure
preprocessed_data_zip_path = "/content/drive/MyDrive/so-vits_colab_files/sovits_data_folders.zip" #@param {type:"string"}
#@markdown ___


if train_from_scratch:
  !unzip -od /content/so-vits-svc/dataset_raw/{model_name} {raw_data_zip_path}
  clear_output()
  print("Training option: train a model from scratch")
else:
  pass

if resume_training_from_checkpoint:
  !unzip -d /content/so-vits-svc {preprocessed_data_zip_path}
  clear_output()
  print("Training option: resume training from latest checkpoint")
else:
  pass

print("|")
print("|")
print("|")
print("Done!")

In [ ]:
#@title #2.1 Start Preprocessing

#@markdown Run this cell either way, even if you already preprocessed your data

%cd /content/so-vits-svc

if train_from_scratch:
  !python resample.py
  clear_output()
  !python preprocess_flist_config.py
  clear_output()
  !python preprocess_hubert_f0.py
  clear_output()
  sovits_data_dir = "/content/drive/MyDrive/so-vits_colab_files"
  if not os.path.exists(sovits_data_dir):
    os.makedirs(sovits_data_dir)
  !zip -r configs dataset
  !cp configs/config.json {sovits_data_dir}
  clear_output()
  print(f"Necessary data folders zipped and saved to {sovits_data_dir}!")
else:
  pass

if resume_training_from_checkpoint:
  print("You already have the preprocessed files!")
else:
  pass

print("|")
print("|")
print("|")
print("Check the configs folder and dataset folder inside so-vits-svc to make sure you are good to go before start training")
print(".... Or you can just ignore this message")

In [ ]:
import os
#@title #3.0 Start training
#@markdown ####Use custom save directory
#@markdown This is recommended so you won't lose your progress if colab decides to kick you

#@markdown This section also determine if you gonna resume training from latest checkpoint or not

#@markdown (Automatically resume if lastest checkpoints are in the folder)
use_custom_save_directory = True #@param {type:"boolean"}
model_save_directory = path-to-custom-model-save-directory #@param {type:"string"}g
if use_custom_save_directory:
  target_string = '  model_dir = os.path.join("./logs", args.model)\n'
  replacement_string = f'  model_dir = os.path.join("{model_save_directory}", args.model)\n'
  with open("/content/so-vits-svc/utils.py", "r") as f:
    lines = f.readlines()
  target_line_number = None
  for i, line in enumerate(lines):
    if line == target_string:
      target_line_number = i
      break
  if target_line_number is None:
      print(f"Error: could not find target string '{target_string}' in utils.py")
  else:
    lines[target_line_number] = replacement_string
    with open("/content/so-vits-svc/utils.py", "w") as f:
      f.writelines(lines)
#@markdown Display tensorboard for the training progress visualization
tensorboard = True  #@param {type:"boolean"}
if tensorboard:
  %load_ext tensorboard
  %tensorboard --logdir {model_save_directory}
print(f"Your model will be saved at {model_save_directory}")
!python train.py -c configs/config.json -m 44k

# **Inference Section**
upload your audio into "raw" folder inside so-vits-svc root directory

In [ ]:
#@title Start inference

#@markdown Parameters see [README.MD#inference](https://github.com/svc-develop-team/so-vits-svc#inference)

#@markdown

wav_filename = "input_wav.wav"  #@param {type:"string"}
model_path = "path-to-model"  #@param {type:"string"}
model_name = "your-model-name"  #@param {type:"string"}
config_path = "path-to-config"  #@param {type:"string"}
trans = "0"  #@param {type:"string"}
cluster_infer_ratio = "0"  #@param {type:"string"}

#@markdown

#@markdown Generally keep default:

slice_db = "-40"  #@param {type:"string"}
wav_format = "flac"  #@param {type:"string"}
wav_output = "/content/so-vits-svc/results/" + wav_filename + "_" + trans + "key" + "_" + speaker + "." + wav_format

!python inference_main.py -n {wav_filename} -m {model_path} -c {config_path} -s {model_name} -t {trans} -cr {cluster_infer_ratio} -sd {slice_db} -wf {wav_format}

#@markdown

#@markdown If you dont want to download from here, uncheck this.
download_after_inference = False  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)